In [ ]:
# === Импорты и установка окружения ===
!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .
!pip install googletrans==4.0.0-rc1 matplotlib scikit-learn imageio ipywidgets torch

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from googletrans import Translator
#from google.colab import files
import imageio
import os
import matplotlib.pyplot as plt
import numpy as np
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# === Основной класс генератора ===

class ShapE3DGenerator:
   

    def __init__(self):
        """Инициализация генератора: загрузка моделей и настройка устройства."""
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"✅ Устройство выбрано: {self.device.type.upper()}")

        # Загрузка моделей
        logger.info("Загрузка моделей...")
        self.transmitter = load_model('transmitter', device=self.device)
        self.text_model = load_model('text300M', device=self.device)
        self.diffusion_process = diffusion_from_config(load_config('diffusion'))
        logger.info("✅ Модели успешно загружены.")

        # Инициализация переводчика
        self.translator = Translator()

        # Хранение состояния
        self.current_latents = None
        self.generated_gifs = []
        self.last_prompt = ""

    def translate_prompt(self, russian_text: str) -> str:
        """
        Переводит текстовый запрос с русского языка на английский.
        :param russian_text: Текст на русском языке.
        :return: Переведенный текст на английском.
        """
        try:
            translation = self.translator.translate(russian_text, src='ru', dest='en')
            return translation.text
        except Exception as e:
            logger.error(f"❌ Ошибка перевода: {e}")
            return russian_text
        
    def generate_3d_model(self,
                          prompt: str,
                          guidance_scale: float = 15.0,
                          batch_size: int = 1,
                          diffusion_steps: int = 64,
                          render_mode: str = 'nerf',
                          image_size: int = 64) -> None:
        """
        Генерирует 3D модель на основе текстового запроса.
        """
        # Округление размера до ближайшего кратного 8
        image_size = ((image_size + 7) // 8) * 8
        logger.info(f"🚀 Начинаем генерацию для запроса: '{prompt}'")

        # Перевод запроса
        english_prompt = self.translate_prompt(prompt)
        self.last_prompt = english_prompt
        logger.info(f"🌐 Переведенный запрос: '{english_prompt}'")

        # Генерация латентных представлений
        try:
            self.current_latents = sample_latents(
                batch_size=batch_size,
                model=self.text_model,
                diffusion=self.diffusion_process,
                guidance_scale=guidance_scale,
                model_kwargs=dict(texts=[english_prompt] * batch_size),
                progress=True,
                clip_denoised=True,
                use_fp16=True,
                use_karras=True,
                karras_steps=diffusion_steps,
                sigma_min=1e-3,
                sigma_max=160,
                s_churn=0,
            ).to(self.device)

            logger.info(f"✅ Латентные представления сгенерированы. Количество: {batch_size}")

            # Создание камер
            cameras = create_pan_cameras(image_size, self.device)

            # Декодирование и визуализация
            self.generated_gifs = []
            for i, latent in enumerate(self.current_latents):
                images = decode_latent_images(
                    self.transmitter,
                    latent.to(self.device),
                    cameras,
                    rendering_mode=render_mode
                )
                self.generated_gifs.append(images)
                display(gif_widget(images))
                logger.info(f"🖼️  GIF #{i+1} сгенерирован и отображен.")

        except Exception as e:
            logger.error(f"❌ Ошибка при генерации: {e}")  

    def save_generated_gif(self, file_name: str = "generated_model.gif") -> None:
        """
        Сохраняет последний сгенерированный GIF.
        """
        if not self.generated_gifs:
            logger.warning("⚠️ Нет сгенерированных GIF для сохранения.")
            return

        try:
            imageio.mimsave(file_name, self.generated_gifs[0], fps=10)
            logger.info(f"💾 GIF успешно сохранен как '{file_name}'")
        except Exception as e:
            logger.error(f"❌ Ошибка при сохранении GIF: {e}")

    def visualize_latent_space(self) -> None:
        """
        Визуализирует распределение значений в латентном пространстве.
        """
        if self.current_latents is None:
            logger.warning("⚠️ Латентное пространство не сгенерировано. Сначала запустите генерацию.")
            return

        latents_np = self.current_latents.cpu().numpy()
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        fig.suptitle('Анализ Латентного Пространства', fontsize=16)

        # Гистограмма всех значений
        axes[0, 0].hist(latents_np.flatten(), bins=50, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Общее Распределение')
        axes[0, 0].set_xlabel('Значение')
        axes[0, 0].set_ylabel('Частота')

        # Гистограммы первых 3 компонентов
        for i in range(min(3, latents_np.shape[1])):
            axes[i//2, i%2].hist(latents_np[:, i], bins=50, color='lightcoral', edgecolor='black')
            axes[i//2, i%2].set_title(f'Компонент {i+1}')
            axes[i//2, i%2].set_xlabel('Значение')
            axes[i//2, i%2].set_ylabel('Частота')

        # Убирает лишние подграфики
        if latents_np.shape[1] < 4:
            for j in range(latents_np.shape[1], 4):
                fig.delaxes(axes[j//2, j%2])

        plt.tight_layout()
        plt.show()
    
    # === Интерфейс пользователя ===

def create_user_interface(generator: ShapE3DGenerator):
    """
    Создает графический интерфейс пользователя.
    """
    # --- Виджеты ввода ---
    prompt_input = widgets.Text(value='красный самолет', description='Запрос:', layout=widgets.Layout(width='auto'))
    guidance_slider = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.1, description='Масштаб:', layout=widgets.Layout(width='auto'))
    render_dropdown = widgets.Dropdown(options=['nerf', 'stf'], value='nerf', description='Рендер:', layout=widgets.Layout(width='auto'))
    size_slider = widgets.IntSlider(value=64, min=32, max=256, step=8, description='Размер:', layout=widgets.Layout(width='auto'))
    steps_slider = widgets.IntSlider(value=64, min=1, max=256, step=1, description='Шаги:', layout=widgets.Layout(width='auto'))
    batch_slider = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Пакет:', layout=widgets.Layout(width='auto'))

    # --- Кнопки ---
    generate_button = widgets.Button(description="Генерировать", button_style='success', layout=widgets.Layout(width='auto'))
    save_button = widgets.Button(description="Сохранить GIF", button_style='info', layout=widgets.Layout(width='auto'))
    histogram_button = widgets.Button(description="Анализ латентных", button_style='warning', layout=widgets.Layout(width='auto'))

    # --- Прогресс-бар ---
    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='Прогресс:', bar_style='info', layout=widgets.Layout(width='auto'))

    # --- Область вывода ---
    output_area = widgets.Output(layout=widgets.Layout(border='solid 1px #ccc', padding='10px'))

    # --- Функции обработки кнопок ---
    def on_generate_clicked(b):
        with output_area:
            clear_output(wait=True)
            progress_bar.value = 0
            progress_bar.description = "Генерация..."
            progress_bar.bar_style = 'info'

            prompt = prompt_input.value.strip()
            if not prompt:
                logger.warning("⚠️ Пустой запрос!")
                return

            try:
                generator.generate_3d_model(
                    prompt=prompt,
                    guidance_scale=guidance_slider.value,
                    batch_size=batch_slider.value,
                    diffusion_steps=steps_slider.value,
                    render_mode=render_dropdown.value,
                    image_size=size_slider.value
                )
                progress_bar.value = 100
                progress_bar.description = "Готово!"
                progress_bar.bar_style = 'success'
            except Exception as e:
                logger.error(f"❌ Ошибка: {e}")
                progress_bar.description = "Ошибка!"
                progress_bar.bar_style = 'danger'

    def on_save_clicked(b):
        with output_area:
            clear_output(wait=True)
            generator.save_generated_gif()

    def on_histogram_clicked(b):
        with output_area:
            clear_output(wait=True)
            generator.visualize_latent_space()

    # --- Привязка событий ---
    generate_button.on_click(on_generate_clicked)
    save_button.on_click(on_save_clicked)
    histogram_button.on_click(on_histogram_clicked)

    # --- Компоновка интерфейса ---
    input_group = widgets.VBox([
        widgets.HBox([prompt_input]),
        widgets.HBox([guidance_slider, render_dropdown]),
        widgets.HBox([size_slider, steps_slider]),
        widgets.HBox([batch_slider])
    ], layout=widgets.Layout(border='solid 1px gray', padding='10px'))

    button_group = widgets.HBox([
        generate_button,
        save_button,
        histogram_button
    ], layout=widgets.Layout(padding='5px'))

    # Отображение  интерфейса
    display(input_group, button_group, progress_bar, output_area)

# === Запуск ===

if __name__ == "__main__":
    shap_e_gen = ShapE3DGenerator()
    create_user_interface(shap_e_gen)
    print("🎉 Генератор 3D моделей Shap-E готов к работе!")
    print("👉 Введите запрос на русском языке и нажмите 'Генерировать'.")
  